In [ ]:
import os
import numpy as np
import pandas as pd

import cv2
from time import time
from importlib import reload
import utils; reload(utils)

import keras

In [ ]:
# valid_sizes = {'Motorola-Droid-Maxx':[(2432,4320),(4320,2432)],
#                'LG-Nexus-5x':[(3024,4032),(4032,3024)],
#                'Sony-NEX-7':[(4000,6000),(6000,4000)],
#                'Motorola-X':[(3120,4160),(4160,3120),(2340,4160),(4160,2340)],
#                'iPhone-6':[(2448,3264),(3264,2448)],
#                'Samsung-Galaxy-S4':[(2322,4128),(4128,2322)],
#                'iPhone-4s':[(2448,3264),(3264,2448)],
#                'Samsung-Galaxy-Note3':[(2322,4128),(4128,2322)],
#                'Motorola-Nexus-6':[(3120,4160),(4160,3120),(3088,4160),(4160,3088)],
#                'HTC-1-M7':[(1520,2688),(2688,1520)]
#               }

In [ ]:
train_path = 'train'
test_path = 'test'
validation_path = 'validation'

In [ ]:
cameras = pd.read_csv('train.csv').camera.unique()

df_train = pd.read_csv('data/train_team_final_v5.csv')

df_cv_test = pd.read_csv('HOLDOUT_VALIDATION/Final/labels.csv')
df_cv_test = df_cv_test[df_cv_test.is_altered == 1].copy().reset_index(drop=True)

# Pseudo Labels Training
# voting = pd.read_csv('Voting_stats_v7_corr.csv')
# voting['fname'] = [os.path.join('test',x) for x in voting.fname]
# voting = voting[['fname','best_camera', 'is_manip','votes']]
# voting.columns = ['fname','camera', 'is_manip', 'votes']
# df_train= voting[(voting.votes == 6)&(voting.is_manip == 'unalt')].copy().reset_index(drop=True)
# df_cv_test= voting[(voting.votes != 6)&(voting.is_manip == 'unalt')].copy().reset_index(drop=True)

EPOCHS = 1000

CATEGORIES = {k: i for i, k in enumerate(cameras)}
df_train['label'] = df_train.camera.map(CATEGORIES)
df_cv_test['label'] = df_cv_test.camera.map(CATEGORIES)

In [ ]:
df_train.head()

In [ ]:
df_cv_test.head()

In [ ]:
# PATH_TO_CV = 'CV_splits_team/'
# N_CV_FOLDS = 3
#utils.make_cv_split(df_train,cameras,'id',N_CV_FOLDS,PATH_TO_CV,seed=24)

In [ ]:
df_train.shape

In [ ]:
df_cv_test.shape

# Train

In [ ]:
import utils; reload(utils)
cv_scores = []
all_preds_test = {}
model_log = 'VGG_16_pretrained_384_ext_stage2_unalt_final'
#for i in range(N_CV_FOLDS):
for i in range(1):
    t1 = time()
    predictions_list = []
    
    print('*'*8,'CV_FOLD_{}'.format(i),'*'*8)
#     df_cv_train = pd.read_csv(os.path.join(PATH_TO_CV,'CV_{}_train.csv'.format(i)))
#     df_cv_test = pd.read_csv(os.path.join(PATH_TO_CV,'CV_{}_test.csv'.format(i)))
#     df_cv_train = df_cv_train.merge(df_train)
#     df_cv_test = df_cv_test.merge(df_train)

    # Pseudo Labels
    #df_cv_train = df_train[df_train.is_external == -1].copy().reset_index(drop=True)
    df_cv_train = df_train.copy().reset_index(drop=True)

    df_cv_train = df_cv_train.sample(frac=1)
    df_cv_test = df_cv_test.sample(frac=1)
    
    #print(pd.crosstab(df_cv_train.is_external, df_cv_train.camera))
    print(df_cv_train.shape)
    print(df_cv_test.shape)
    
    model_name = 'VGG16'
    preprocessing = utils.get_preprocess(model_name)

    TRAIN_BATCH_SIZE = utils.get_batch_size(model_name)
    EVALUATION_BATCH_SIZE = int(TRAIN_BATCH_SIZE*1.5)
    
    # Calculate number of steps
    steps_per_epoch = df_cv_train.shape[0]//TRAIN_BATCH_SIZE * 4
    validation_steps = df_cv_test.shape[0]//EVALUATION_BATCH_SIZE + 1
    
    # ****************************************************************************************
    # Train HEAD
    weights_path_head = "{}_head_fold_{}.hdf5".format(model_log,i)   
    
    callbacks = [
             keras.callbacks.ModelCheckpoint(weights_path_head, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True)
            ]
    
    # Initialize Model
    model = utils.InitialiazeModel(head_only=True,weights='',model_name = model_name, lr=0.001)
    model.fit_generator(generator = utils.train_batch_generator(df_cv_train.fname.values, df_cv_train.label.values,TRAIN_BATCH_SIZE,preprocessing),
                    validation_data = utils.evaluation_batch_generator(df_cv_test.fname.values, df_cv_test.label.values,EVALUATION_BATCH_SIZE,preprocessing),
                    validation_steps = validation_steps,
                    epochs = 2,
                    steps_per_epoch = steps_per_epoch,
                   max_queue_size=20,
                    callbacks=callbacks,
                        use_multiprocessing=False,
                   verbose = 1)
    
    # ****************************************************************************************
    # Train ALL
    df_cv_train = df_cv_train.sample(frac=1)
    df_cv_test = df_cv_test.sample(frac=1)
    
    # Callbacks
    LOG_NAME = "{}_0001_fold_{}".format(model_log,i)
    #weights_path = "VGG_16_pretrained_ext_00001_manip.{epoch:02d}-{val_loss:.4f}.hdf5"
    weights_path = "{}_0001_fold_{}.hdf5".format(model_log,i)   
    weights_path_head = "{}_head_fold_{}.hdf5".format(model_log,i)
    
    model = utils.InitialiazeModel(head_only=False,weights=weights_path_head,model_name = model_name, lr=0.001)
    
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0),
             keras.callbacks.ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=False, verbose=0,
                                             save_weights_only=True),
             keras.callbacks.TensorBoard(log_dir=os.path.join('logs/',LOG_NAME))
            ]
    
    model.fit_generator(generator = utils.train_batch_generator(df_cv_train.fname.values, df_cv_train.label.values,TRAIN_BATCH_SIZE,preprocessing),
                    validation_data = utils.evaluation_batch_generator(df_cv_test.fname.values, df_cv_test.label.values,EVALUATION_BATCH_SIZE,preprocessing),
                    validation_steps = validation_steps,
                    epochs = EPOCHS,
                    steps_per_epoch = steps_per_epoch,
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)
    
    # ****************************************************************************************
        # Train ALL
#     df_cv_train = df_cv_train.sample(frac=1)
#     df_cv_test = df_cv_test.sample(frac=1)
    
#     # Callbacks
#     LOG_NAME = "{}_00001_fold_{}".format(model_log,i)
#     #weights_path = "VGG_16_pretrained_ext_00001.{epoch:02d}-{val_loss:.4f}.hdf5"
#     weights_path = "{}_00001_fold_{}.hdf5".format(model_log,i) 
#     weights_path_head = "{}_0001_fold_{}.hdf5".format(model_log,i) 
# ERROR
    
#     model = utils.InitialiazeModel(head_only=False,weights=weights_path_head,model_name = model_name, lr=0.0001)
    
#     callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0),
#              keras.callbacks.ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, verbose=0,
#                                              save_weights_only=True),
#              keras.callbacks.TensorBoard(log_dir=os.path.join('logs/',LOG_NAME))
#             ]

#     model.fit_generator(generator = utils.train_batch_generator(df_cv_train.fname.values, df_cv_train.label.values,TRAIN_BATCH_SIZE,preprocessing),
#                     validation_data = utils.evaluation_batch_generator(df_cv_test.fname.values, df_cv_test.label.values,EVALUATION_BATCH_SIZE,preprocessing),
#                     validation_steps = validation_steps,
#                     epochs = EPOCHS,
#                     steps_per_epoch = steps_per_epoch,
#                    max_queue_size=20,
#                    callbacks=callbacks,
#                    verbose = 1)
#     # ****************************************************************************************
#         # Train ALL
#     df_cv_train = df_cv_train.sample(frac=1)
#     df_cv_test = df_cv_test.sample(frac=1)
    
#     # Callbacks
#     LOG_NAME = "{}_000001_fold_{}".format(model_log,i)
#     #weights_path = "VGG_16_pretrained_ext_00001.{epoch:02d}-{val_loss:.4f}.hdf5"
#     weights_path = "{}_000001_fold_{}.hdf5".format(model_log,i)   
#     weights_path_head = "{}_00001_fold_{}.hdf5".format(model_log,i) 
    
#     model = utils.InitialiazeModel(head_only=False,weights=weights_path_head,model_name = model_name, lr=0.00001)
    
#     callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0),
#              keras.callbacks.ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, verbose=0,
#                                              save_weights_only=True),
#              keras.callbacks.TensorBoard(log_dir=os.path.join('logs/',LOG_NAME)),
#                 keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001, verbose=1)
#             ]
# #     callbacks = [
# #              #keras.callbacks.LearningRateScheduler(utils.step_decay)
# #             ]

#     model.fit_generator(generator = utils.train_batch_generator(df_cv_train.fname.values, df_cv_train.label.values,TRAIN_BATCH_SIZE,preprocessing),
#                     validation_data = utils.evaluation_batch_generator(df_cv_test.fname.values, df_cv_test.label.values,EVALUATION_BATCH_SIZE,preprocessing),
#                     validation_steps = validation_steps,
#                     epochs = EPOCHS,
#                     steps_per_epoch = steps_per_epoch,
#                    max_queue_size=20,
#                    callbacks=callbacks,
#                    verbose = 1)
# learning rate schedule
# def step_decay(epoch):
#     initial_lrate = 0.001
#     drop = 0.5
#     epochs_drop = 2
#     lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
#     return lrate
# #keras.callbacks.LearningRateScheduler(step_decay)
    
    
    print('Time for fold {}: {}'.format(i,time()-t1))

# Validation

In [ ]:
# External Validation -------------------------------
df_cv_test = pd.read_csv('external_validation.csv')
df_cv_test['fname'] = [os.path.join('external_validation/',fname) for fname in df_cv_test.fname]

In [ ]:
import utils; reload(utils)

testing = True
model_name = 'VGG16'
#weights_path = "Final_Model_Manip.hdf5"
weights_path = "Final_Model_Unalt.hdf5"

TTA = True
augment_number_per_image = 36

cv_scores = []
all_preds_test = {}
#for i in range(N_CV_FOLDS):
for i in range(1):
    predictions_list = []
    predictions_list2 = []
    
    print('*'*8,'CV_FOLD_{}'.format(i),'*'*8)
    if testing:
        # Testing Set
        test_images = []
        for fname in sorted(os.listdir(test_path)):
            if 'manip' in fname:
                continue
            test_images.append(os.path.join(test_path,fname))
        df_cv_test = pd.DataFrame(test_images, columns=['fname'])
    else:
        print('1')
        #df_cv_test = pd.read_csv(os.path.join(PATH_TO_CV,'CV_{}_test.csv'.format(i)))
        #df_cv_test = df_cv_test.merge(df_train)
    
    probs, preds = utils.model_predict(df_cv_test.fname.values,model_name,weights_path,TTA,augment_number_per_image)
    
    df_cv_test['pred'] = preds

In [ ]:
df_cv_test.head()

In [ ]:
df_cv_test.shape

In [ ]:
# save best probs to the file
# to_save = pd.DataFrame(probs)
# cameras = pd.read_csv('train.csv').camera.unique()
# CATEGORIES_names = {i: k for i, k in enumerate(cameras)}
# to_save.columns = pd.Series([int(x) for x in to_save.columns.values]).map(CATEGORIES_names).values
# to_save['fname'] = [x.split('/')[1] for x in df_cv_test['fname']]
# to_save.to_csv('Final_Model_Probs_Unalt.csv',index=False)

In [ ]:
# Mean of multiple models
# from scipy.stats.mstats import gmean
# probs = np.mean(np.array([probs0,probs]), axis=0)
# #preds_camera_ens = gmean(np.array([probs0,probs]), axis=0)
# df_cv_test['pred'] = np.argmax(np.array(probs0),axis=1) 

In [ ]:
# Logloss on validation
from sklearn.metrics import log_loss
y_true = [np.argmax(x == cameras) for x in df_cv_test[df_cv_test.is_altered == 0].camera]
log_loss(y_true, probs[df_cv_test.is_altered == 0], eps=1e-15)

In [ ]:
df_cv_test['pred_camera'] = cameras[df_cv_test.pred]

In [ ]:
df_cv_test.pred_camera.value_counts(normalize=True)

In [ ]:
np.mean(df_cv_test.camera == df_cv_test.pred_camera)

In [ ]:
np.mean(df_cv_test[df_cv_test.is_altered == 0].camera == df_cv_test[df_cv_test.is_altered == 0].pred_camera) * 0.7 + np.mean(df_cv_test[df_cv_test.is_altered == 1].camera == df_cv_test[df_cv_test.is_altered == 1].pred_camera) * 0.3

In [ ]:
df_cv_test['is_manip'] = df_cv_test.fname.str[-9:-4]
df_cv_test.head()

In [ ]:
df['balanced_pred'] = -1

ix = df.fname.str.contains('manip').index
while np.any(df.iloc[ix]['balanced_pred'] == -1):
    for c in list(df[df.is_manip == 'manip'].camera.value_counts(normalize=True).index[::-1]):
        idx = df[df.iloc[ix]['balanced_pred'] == -1][c].argmax()
        df.loc[idx,'balanced_pred'] = c

ix = df.fname.str.contains('unalt').index
while np.any(df.iloc[ix]['balanced_pred'] == -1):
    for c in list(df[df.is_manip == 'unalt'].camera.value_counts(normalize=True).index[::-1]):
        idx = df[df.iloc[ix]['balanced_pred'] == -1][c].argmax()
        df.loc[idx,'balanced_pred'] = c

In [ ]:
df_cv_test['balanced_pred'] = df.balanced_pred

In [ ]:
df_cv_test[df_cv_test.pred != df_cv_test.balanced_pred].shape

In [ ]:
df_cv_test.balanced_pred.value_counts()

In [ ]:
df_cv_test['max_prob'] = np.max(probs,axis=1)

In [ ]:
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)
test = pd.DataFrame(test_images, columns=['fname'])
test['camera'] = cameras[df_cv_test.pred]

In [ ]:
test.camera.value_counts(normalize=True)

In [ ]:
# TTA-36 balanced
Motorola-Nexus-6        0.104545
iPhone-4s               0.102273
Samsung-Galaxy-S4       0.101515
Motorola-X              0.100000
Motorola-Droid-Maxx     0.099621
iPhone-6                0.099621
Samsung-Galaxy-Note3    0.099242
LG-Nexus-5x             0.098485
HTC-1-M7                0.097727
Sony-NEX-7              0.096970

In [ ]:
test.to_csv('Final_Manip.csv',index=False)